<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/computer-vision/4_Pretrained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory

In [4]:
from google.colab import files
r=files.upload()

Saving kaggle.json to kaggle.json


In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c dogs-vs-cats

 99% 805M/812M [00:04<00:00, 192MB/s]
100% 812M/812M [00:04<00:00, 171MB/s]


In [ ]:
!unzip -qq dogs-vs-cats.zip

In [ ]:
!unzip -qq train.zip

In [13]:
import os, pathlib, shutil

In [14]:
src_dir = pathlib.Path("train")
dst_dir = pathlib.Path("cats-vs-dogs")

In [24]:
def create_directory(name, start_index, end_index):

  for category in ["cat", "dog"]:

    sub_dir = dst_dir / name/ category
    os.makedirs(sub_dir)

    for i in range(start_index, end_index):
      src_file = src_dir / f"{category}.{i}.jpg"
      shutil.copy(src=src_file, dst=sub_dir)

In [25]:
create_directory("train", 0, 1000)
create_directory("validation", 1000, 1500)
create_directory("test", 1500, 2500)

In [27]:
train_dataset = image_dataset_from_directory(dst_dir / "train", image_size = (180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory(dst_dir / "validation", image_size = (180, 180), batch_size=32)
test_dataset = image_dataset_from_directory(dst_dir / "test", image_size = (180, 180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
